# Experiment 04

This Experiment is the main experiment of the work, consists in run an optimized Random Forest in order to try to predict the values in question.(https://www.researchgate.net/publication/221533996_From_an_artificial_neural_network_to_a_stock_market_day-trading_system_A_case_study_on_the_BMF_BOVESPA).

This method consists in using the sliding window 

In [1]:
import numpy as np
import pandas as pd

import Utils 
from Utils import calculateEMA, calculateBB, addPreviousDays, removeColumns, getPeriod, mape, mae, createDate, getDate, printResult

import matplotlib.pyplot as plt
import matplotlib.dates as mpdates
import mplfinance as mpf
from mplfinance.original_flavor import candlestick_ohlc

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor

### Constants and Parameters

In [2]:
dfColumns = ['date','open', 'high', 'low','close','tickvol','volume','spread']
newColumns = ['open', 'high', 'low', 'close']
col15 = ['open','d-5_high','d-5_low','d-4_high','d-4_low','d-3_high','d-3_low','d-2_high','d-2_low','d-1_high','d-1_low','EMA_high','EMA_low','UB_close','LB_close']
col25 = ['open','d-5_open','d-5_high','d-5_low','d-5_close','d-4_open','d-4_high','d-4_low','d-4_close','d-3_open','d-3_high','d-3_low','d-3_close','d-2_open','d-2_high','d-2_low','d-2_close','d-1_open','d-1_high','d-1_low','d-1_close','EMA_high' ,'EMA_low','UB_close','LB_close']
DAYS = 5

In [3]:
TEST_BEGIN = '2020.06.01'
TEST_END = '2021.06.01'

WINDOW = 128
DRIFT = 10

TIMES = 1

def safeDate(df, begin, period):
    index = df[df['date'] == begin].index[0]
    
    newIndex = index - period*3
    newDate = df[df.index == newIndex].date
    
    return newDate.iloc[0]

### Opening the dataset

In [4]:
df = pd.read_csv('../Data/PETR4_Daily.csv', sep = "\t")
df.columns = dfColumns
df = removeColumns(df, ['tickvol', 'volume','spread'])

beginDate = safeDate(df, TEST_BEGIN, WINDOW)

df2 = pd.read_csv('../Data/VALE3_Daily.csv', sep = "\t")
df2.columns = dfColumns
df2 = removeColumns(df2, ['tickvol', 'volume','spread'])

In [5]:
#PETR4
df = calculateEMA(df,DAYS,newColumns)
df = calculateBB(df,DAYS,newColumns)
df = addPreviousDays(df,DAYS, newColumns)

#VALE3
df2 = calculateEMA(df2,DAYS,newColumns)
df2 = calculateBB(df2,DAYS,newColumns)
df2 = addPreviousDays(df2,DAYS, newColumns)

In [6]:
initialDf_PETR4 = getPeriod(df, beginDate ,TEST_END, True)
initialDf_VALE3 = getPeriod(df2,beginDate,TEST_END, True)

In [7]:
initialDf_PETR4

,date,open,high,low,close,EMA_open,EMA_high,EMA_low,EMA_close,UB_open,...,d-3_low,d-3_close,d-4_open,d-4_high,d-4_low,d-4_close,d-5_open,d-5_high,d-5_low,d-5_close
0,2018.11.07,25.14,25.17,23.91,23.94,25.039658,25.453964,24.326933,24.631140,25.867302,...,24.68,24.87,25.73,25.79,24.85,25.15,24.31,25.49,24.05,25.49
1,2018.11.08,24.40,24.61,23.08,23.08,24.826439,25.172642,23.911289,24.114093,25.617200,...,24.95,25.64,25.27,25.51,24.68,24.87,25.73,25.79,24.85,25.15
2,2018.11.09,22.94,23.42,22.47,23.18,24.197626,24.588428,23.430859,23.802729,26.280632,...,24.57,24.75,25.04,25.64,24.95,25.64,25.27,25.51,24.68,24.87
3,2018.11.12,23.25,23.60,23.02,23.06,23.881751,24.258952,23.293906,23.555153,26.065595,...,23.91,23.94,24.85,25.77,24.57,24.75,25.04,25.64,24.95,25.64
4,2018.11.13,23.05,23.05,21.80,22.07,23.604500,23.855968,22.795937,23.060102,25.692626,...,23.08,23.08,25.14,25.17,23.91,23.94,24.85,25.77,24.57,24.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,2021.05.26,25.91,26.18,25.65,26.09,26.114791,26.354219,25.760657,26.042263,26.608719,...,25.94,25.95,26.15,26.28,25.72,25.93,25.85,26.35,25.80,26.15
629,2021.05.27,26.08,26.25,25.77,25.91,26.103194,26.319479,25.763771,25.998175,26.604087,...,25.99,26.39,26.08,26.31,25.94,25.95,26.15,26.28,25.72,25.93
630,2021.05.28,26.20,27.02,26.15,26.99,26.135462,26.552986,25.892514,26.328784,26.617551,...,25.78,25.84,26.28,26.52,25.99,26.39,26.08,26.31,25.94,25.95
631,2021.05.31,26.96,27.20,26.78,26.87,26.410308,26.768657,26.188343,26.509189,27.147657,...,25.65,26.09,26.48,26.62,25.78,25.84,26.28,26.52,25.99,26.39


In [8]:
initialDf_VALE3

,date,open,high,low,close,EMA_open,EMA_high,EMA_low,EMA_close,UB_open,...,d-3_low,d-3_close,d-4_open,d-4_high,d-4_low,d-4_close,d-5_open,d-5_high,d-5_low,d-5_close
0,2018.11.07,52.83,53.16,52.48,52.65,52.064126,52.757230,51.445483,52.138492,54.359077,...,50.76,52.57,49.69,51.85,49.62,51.85,49.00,50.15,48.42,49.25
1,2018.11.08,52.82,53.73,52.13,52.13,52.316084,53.081487,51.673655,52.135661,53.339200,...,52.15,52.60,51.88,53.26,50.76,52.57,49.69,51.85,49.62,51.85
2,2018.11.09,51.65,52.11,49.49,49.96,52.094056,52.757658,50.945770,51.410441,53.455000,...,51.71,52.32,52.67,53.03,52.15,52.60,51.88,53.26,50.76,52.57
3,2018.11.12,50.37,50.69,49.73,50.15,51.519371,52.068439,50.540513,50.990294,53.991466,...,52.48,52.65,52.11,52.89,51.71,52.32,52.67,53.03,52.15,52.60
4,2018.11.13,50.19,51.44,49.69,51.44,51.076247,51.858959,50.257009,51.140196,54.121682,...,52.13,52.13,52.83,53.16,52.48,52.65,52.11,52.89,51.71,52.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,2021.05.26,106.37,110.32,106.03,110.20,109.166674,111.131617,107.622606,109.807900,113.973861,...,108.50,109.40,111.70,112.10,110.20,111.11,112.10,112.78,109.84,112.25
629,2021.05.27,110.80,112.09,109.77,110.99,109.711116,111.451078,108.338404,110.201933,113.405981,...,107.51,109.78,111.11,111.25,108.50,109.40,111.70,112.10,110.20,111.11
630,2021.05.28,111.11,112.75,109.42,111.59,110.177411,111.884052,108.698936,110.664622,113.405981,...,106.78,107.05,108.00,110.14,107.51,109.78,111.11,111.25,108.50,109.40
631,2021.05.31,112.50,114.89,112.50,114.78,110.951607,112.886035,109.965957,112.036415,114.811204,...,106.03,110.20,110.28,110.89,106.78,107.05,108.00,110.14,107.51,109.78


#### Experimento 04: Utilizando Random Forest

In [9]:
def getTrainPeriod(df, begin, period):
    
    index = df[df['date'] == begin].index[0]
    index = index-1
    
    endDate = df[df.index == index].date
    beginDate = df[df.index == (index-period)].date
    
    return beginDate.iloc[0], endDate.iloc[0]
    

def exp04(df, stock, inputColumns = []):
    
    beginTrain, endTrain = getTrainPeriod(df,TEST_BEGIN, WINDOW)
    
    df_test_aux = getPeriod(df, TEST_BEGIN, TEST_END)
    
    indexBegin = df_test_aux.head(1).index[0]
    indexEnd = df_test_aux.tail(1).index[0]
    
    for i in range (indexBegin, indexEnd, DRIFT):
        
        df_test = df[(df.index >= i) & (df.index <= i+DRIFT-1)]
        df_train = df[(df.index >= i-WINDOW) & (df.index < i)]
    
        if(inputColumns == []):
            x_train = df_train.drop(['date', 'high', 'low', 'close'], axis=1)
            y_train = df_train[['high','low']]

            x_test = df_test.drop(['date', 'high', 'low', 'close'], axis=1)
            y_test = df_test[['high','low']]
        else:
            x_train = df_train[inputColumns]
            y_train = df_train[['high','low']]

            x_test = df_test[inputColumns]
            y_test = df_test[['high','low']]

        rf = RandomForestRegressor(n_estimators = 500)
        rf.fit(x_train, y_train);
        y_pred = rf.predict(x_test)
        
        #print('train: ' + df_train['date'].head(1).tolist()[0] + ' - ' + df_train['date'].tail(1).tolist()[0])
        #print('_test: ' + df_test['date'].head(1).tolist()[0] + ' - ' + df_test['date'].tail(1).tolist()[0])
        #printResult(mape(y_test['high'],y_pred[:,0]), 'High')
        #printResult(mape(y_test['low'],y_pred[:,1]), 'Low')
        #print('###')
        
        if(i == indexBegin):
            #max_test = np.array(y_test['high'])
            #min_test = np.array(y_test['low'])
            #max_pred = np.array(y_pred[:,0])
            #min_pred = np.array(y_pred[:,1])
            
            predDf = y_test.copy()
            predDf['high_pred'] = y_pred[:,0]
            predDf['low_pred'] = y_pred[:,1]
        
        else:
            #max_test = np.append(max_test, np.array(y_test['high']))
            #min_test = np.append(min_test, np.array(y_test['low']))
            #max_pred = np.append(max_pred, np.array(y_pred[:,0]))
            #min_pred = np.append(min_pred, np.array(y_pred[:,1]))
            
            auxDf = y_test.copy()
            auxDf['high_pred'] = y_pred[:,0]
            auxDf['low_pred'] = y_pred[:,1]
            predDf = predDf.append(auxDf)
        
        #errorMax = mape(predDf['high'],predDf['high_pred'])
        #errorMin = mape(predDf['low'],predDf['low_pred'])
    
    return predDf

def test30timesExp04(df,stock, phrase, columnInput = []):

    acc_predDf = pd.DataFrame()
    
    for i in range(1, TIMES+1):
        if(columnInput == []):
            predDf = exp04(df, stock)
        else: 
            predDf = exp04(df, stock, columnInput)
        
        acc_predDf = acc_predDf.append(predDf)
        
    print(phrase)
    printResult(mape(acc_predDf['high'],acc_predDf['high_pred']), 'High')
    printResult(mape(acc_predDf['low'],acc_predDf['low_pred']), 'Low')
    
    return acc_predDf

In [10]:
test30timesExp04(initialDf_PETR4, 'PETR4', 'PETR4 15 Variables', col15)
test30timesExp04(initialDf_PETR4, 'PETR4', 'PETR4 25 Variables', col25)
test30timesExp04(initialDf_PETR4, 'PETR4', 'PETR4 33 Variables')
print('\n')
test30timesExp04(initialDf_VALE3, 'VALE3', 'VALE3 15 Variables', col15)
test30timesExp04(initialDf_VALE3, 'VALE3', 'VALE3 25 Variables', col25)
test30timesExp04(initialDf_VALE3, 'VALE3', 'VALE3 33 Variables')

PETR4 15 Variables
High: 2.16 +- 2.4 %
Low: 2.16 +- 2.76 %
PETR4 25 Variables
High: 2.22 +- 2.44 %
Low: 2.21 +- 2.81 %
PETR4 33 Variables
High: 2.18 +- 2.44 %
Low: 2.2 +- 2.81 %


VALE3 15 Variables
High: 2.35 +- 2.81 %
Low: 2.4 +- 2.7 %
VALE3 25 Variables
High: 2.37 +- 2.81 %
Low: 2.42 +- 2.71 %
VALE3 33 Variables
High: 2.39 +- 2.78 %
Low: 2.45 +- 2.71 %


,high,low,high_pred,low_pred
384,50.40,49.14,49.70354,48.56770
385,50.35,49.60,50.44036,49.50986
386,50.86,49.96,50.68162,49.82192
387,52.13,50.09,50.98270,50.08858
388,53.07,50.86,52.30380,51.27000
...,...,...,...,...
628,110.32,106.03,110.34418,107.92394
629,112.09,109.77,111.46722,108.97578
630,112.75,109.42,111.60596,109.15234
631,114.89,112.50,113.88220,110.94138


In [22]:
predDf_petr4 = test30timesExp04(initialDf_PETR4, 'PETR4', 'PETR4 15 Variables', col15)
predDf_vale3 = test30timesExp04(initialDf_VALE3, 'VALE3', 'VALE3 15 Variables', col15)
#acc_predDf['error'] = np.abs(acc_predDf['high']-acc_predDf['high_pred'])/acc_predDf['high']

display(predDf_petr4.head(10))
predDf_petr4.to_pickle('../Data/PETR4_Pred_ytd_RF.pkl')
display(predDf_vale3.head(10))
predDf_vale3.to_pickle('../Data/VALE3_Pred_ytd_RF.pkl')

PETR4 15 Variables
High: 2.16 +- 2.4 %
Low: 2.15 +- 2.77 %
VALE3 15 Variables
High: 2.34 +- 2.8 %
Low: 2.38 +- 2.7 %


,high,low,high_pred,low_pred
384,19.89,19.35,19.44738,18.66368
385,20.70,19.93,19.47542,18.66438
386,21.20,20.71,19.50732,18.68374
387,21.07,20.36,19.49492,18.67182
388,22.28,21.34,19.83712,18.94600
389,21.86,21.29,20.74694,19.73518
390,21.32,20.94,20.30616,19.35290
391,21.19,20.32,20.14790,19.22086
392,20.48,19.45,19.52158,18.69234
393,20.40,18.91,19.49382,18.67248


,high,low,high_pred,low_pred
384,50.40,49.14,49.69660,48.57718
385,50.35,49.60,50.69360,49.63732
386,50.86,49.96,50.86930,49.92300
387,52.13,50.09,51.00598,50.05238
388,53.07,50.86,52.42204,51.35590
389,52.08,50.52,52.23658,51.09356
390,51.47,50.65,52.17516,50.85774
391,51.66,50.31,52.16182,50.75380
392,50.59,48.65,51.76354,50.31544
393,50.94,48.79,50.71086,49.29288
